# 🔪 ETAPA 4: SEGMENTAÇÃO INTELIGENTE EM CHUNKS

## 🎯 **O que esta etapa faz**
Divide o texto extraído em pedaços menores (chunks) de tamanho adequado para processamento de embeddings, preservando o contexto e a semântica.

## 🤔 **Por que esta etapa é necessária**
Textos longos são difíceis de processar eficientemente. Precisamos:
- ✂️ **Dividir em pedaços menores** para modelos de embedding
- 🔗 **Manter contexto** com sobreposição entre chunks
- 📏 **Respeitar limites** de tokens dos modelos (500 tokens)
- 🧠 **Preservar semântica** não cortando no meio de frases

## ⚙️ **Como funciona**
1. **Carrega texto processado** da etapa anterior
2. **Tokeniza inteligentemente** respeitando palavras e frases
3. **Cria chunks** de ~500 tokens com overlap de ~100 tokens
4. **Ajusta limites** para não cortar frases no meio
5. **Preserva metadados** de origem e posição
6. **Valida qualidade** dos chunks gerados

## 📊 **Parâmetros que você pode ajustar**
- `chunk_size`: Tamanho alvo em tokens (padrão: 500)
- `overlap_size`: Sobreposição em tokens (padrão: 100)
- `respect_sentences`: Não cortar frases (padrão: True)
- `min_chunk_size`: Tamanho mínimo válido (padrão: 50)
- `max_chunk_size`: Tamanho máximo permitido (padrão: 750)

## 👁️ **O que esperar de saída**
- 📁 **Pasta `chunks/`** com chunks organizados
- 📄 **Arquivos `*_chunks.json`** com chunks e metadados
- 📊 **Estatísticas** de segmentação por documento
- 🔗 **Mapeamento** de chunks para documentos originais

## ⚠️ **Pontos importantes**
- **Requer Etapa 3** (Processamento Docling) concluída primeiro
- **Tokenização** é aproximada (baseada em espaços)
- **Overlap** garante continuidade de contexto
- **Modo independente** disponível para testes

## 🛡️ Verificação de Dependências

In [ ]:
# 🛡️ VERIFICAÇÃO AUTOMÁTICA DE DEPENDÊNCIAS
import sys
import os
from pathlib import Path
from datetime import datetime

# Adicionar biblioteca ao path
sys.path.insert(0, str(Path().parent / "src"))

from pipeline_utils import (
    PipelineState, 
    check_prerequisites, 
    show_pipeline_progress
)

print("🔪 ETAPA 4: SEGMENTAÇÃO INTELIGENTE EM CHUNKS")
print("=" * 60)
print(f"🕒 Iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Verificar dependências
CURRENT_STAGE = 4
prerequisites_ok = check_prerequisites(CURRENT_STAGE)

if not prerequisites_ok:
    print("\n❌ ERRO: Dependências não atendidas!")
    print("📋 Execute primeiro as etapas anteriores:")
    print("   1. 01_FUNDACAO_PREPARACAO.ipynb")
    print("   2. 02_COLETA_GITLAB.ipynb")
    print("   3. 03_PROCESSAMENTO_DOCLING.ipynb")
    
    show_pipeline_progress()
    raise RuntimeError("Dependências não atendidas - execute etapas anteriores primeiro")

print("\n✅ Dependências verificadas - pode prosseguir")
print("📈 Progresso atual do pipeline:")
show_pipeline_progress()

## 📋 Configuração de Chunking

In [ ]:
# 📋 CONFIGURAÇÃO DOS PARÂMETROS DE CHUNKING
print("📋 Configurando parâmetros de segmentação...")

# 🎯 PARÂMETROS AJUSTÁVEIS
chunking_config = {
    # Tamanhos
    "chunk_size": 500,          # Tokens por chunk
    "overlap_size": 100,        # Overlap entre chunks
    "min_chunk_size": 50,       # Tamanho mínimo válido
    "max_chunk_size": 750,      # Tamanho máximo permitido
    
    # Comportamento
    "respect_sentences": True,   # Não cortar frases
    "respect_paragraphs": False, # Pode cortar parágrafos
    "preserve_structure": True,  # Manter info de estrutura
    
    # Processamento
    "batch_size": 8,             # Documentos processados simultaneamente
    "skip_empty_chunks": True,   # Descartar chunks vazios
    
    # Saída
    "output_dir": "../pipeline_data/chunks",
    "overwrite_existing": True
}

print("\n📊 Parâmetros de chunking configurados:")
print(f"   📏 Tamanho do chunk: {chunking_config['chunk_size']} tokens")
print(f"   🔗 Overlap: {chunking_config['overlap_size']} tokens")
print(f"   📐 Faixa válida: {chunking_config['min_chunk_size']}-{chunking_config['max_chunk_size']} tokens")
print(f"   📝 Respeitar frases: {chunking_config['respect_sentences']}")
print(f"   🏗️ Preservar estrutura: {chunking_config['preserve_structure']}")
print(f"   📦 Lote: {chunking_config['batch_size']} documentos simultâneos")
print(f"   📁 Saída: {chunking_config['output_dir']}")

# Preparar diretório de saída
from pipeline_utils import ensure_directory
ensure_directory(chunking_config["output_dir"])

print("\n✅ Configuração de chunking pronta")

## 📄 Carregamento dos Documentos Processados

In [ ]:
# 📄 CARREGAMENTO DOS DOCUMENTOS PROCESSADOS
print("📄 Carregando documentos da etapa anterior...")

# Carregar dados da etapa 3 (Docling)
state = PipelineState()
try:
    docling_data = state.load_stage_data(3)
    print("✅ Dados carregados da Etapa 3 (Docling)")
    
    processed_docs = docling_data["processed_documents"]
    text_files = docling_data["next_stage_instructions"]["text_files_to_chunk"]
    
    print(f"   📄 Documentos processados: {len(processed_docs)}")
    print(f"   📝 Arquivos de texto: {len(text_files)}")
    print(f"   📁 Diretório origem: {docling_data['next_stage_instructions']['processed_directory']}")
    
except Exception as e:
    print(f"❌ Erro ao carregar dados: {e}")
    raise

# Verificar se arquivos de texto existem
available_text_files = []
for text_file_path in text_files:
    file_path = Path(text_file_path)
    if file_path.exists():
        file_size = file_path.stat().st_size
        if file_size > 0:
            available_text_files.append(text_file_path)
            print(f"   ✅ {file_path.name} ({file_size:,} bytes)")
        else:
            print(f"   ⚠️ {file_path.name} (arquivo vazio)")
    else:
        print(f"   ❌ {file_path.name} (não encontrado)")

if not available_text_files:
    print("\n❌ Nenhum arquivo de texto disponível para chunking")
    raise ValueError("Nenhum arquivo de texto para processar")

print(f"\n📊 Arquivos prontos para chunking: {len(available_text_files)}")
print("\n" + "=" * 60)

## ✂️ Segmentação em Chunks

In [ ]:
# ✂️ EXECUÇÃO DA SEGMENTAÇÃO EM CHUNKS
print("✂️ Iniciando segmentação em chunks...")

from chunking_utils import batch_process_documents_chunking
from pipeline_utils import format_file_size
import json

print(f"\n🚀 Processando {len(available_text_files)} documentos...")
print("=" * 60)

# Executar chunking em lote
start_time = datetime.now()

chunking_results = batch_process_documents_chunking(
    document_list=available_text_files,
    output_dir=chunking_config["output_dir"],
    chunk_size=chunking_config["chunk_size"],
    overlap_size=chunking_config["overlap_size"]
)

end_time = datetime.now()
chunking_duration = (end_time - start_time).total_seconds()

print("\n" + "=" * 60)
print("📊 RESULTADO DA SEGMENTAÇÃO:")

# Contar sucessos e falhas
successful_chunking = [r for r in chunking_results if r["status"] == "success"]
failed_chunking = [r for r in chunking_results if r["status"] == "failed"]

total_chunks = sum(r["chunk_count"] for r in successful_chunking)
total_tokens = sum(r["total_tokens"] for r in successful_chunking)

print(f"   📄 Documentos processados: {len(chunking_results)}")
print(f"   ✅ Sucessos: {len(successful_chunking)}")
print(f"   ❌ Falhas: {len(failed_chunking)}")
print(f"   🔪 Total de chunks: {total_chunks:,}")
print(f"   🎯 Total de tokens: {total_tokens:,}")
print(f"   ⏱️ Tempo total: {chunking_duration:.1f} segundos")

if successful_chunking:
    avg_chunks_per_doc = total_chunks / len(successful_chunking)
    avg_tokens_per_chunk = total_tokens / total_chunks if total_chunks > 0 else 0
    print(f"   📈 Média: {avg_chunks_per_doc:.1f} chunks por documento")
    print(f"   📏 Média: {avg_tokens_per_chunk:.0f} tokens por chunk")

# Mostrar resultados de sucesso
if successful_chunking:
    print("\n📁 Documentos segmentados com sucesso:")
    for result in successful_chunking[:10]:  # Mostrar apenas os primeiros 10
        filename = Path(result["source_document"]).name
        chunks = result["chunk_count"]
        tokens = result["total_tokens"]
        print(f"   ✅ {filename}: {chunks} chunks, {tokens:,} tokens")
    
    if len(successful_chunking) > 10:
        remaining = len(successful_chunking) - 10
        print(f"   ... e mais {remaining} documentos")

# Mostrar falhas se houver
if failed_chunking:
    print(f"\n⚠️ Documentos com problemas ({len(failed_chunking)}):")
    for result in failed_chunking[:5]:  # Mostrar apenas os primeiros 5
        filename = Path(result["source_document"]).name
        error = result["error"]
        print(f"   ❌ {filename}: {error}")
    
    if len(failed_chunking) > 5:
        remaining = len(failed_chunking) - 5
        print(f"   ... e mais {remaining} erros")

print("\n✅ Segmentação concluída!")

## 📊 Validação e Qualidade dos Chunks

In [ ]:
# 📊 VALIDAÇÃO E ANÁLISE DE QUALIDADE DOS CHUNKS
print("📊 Validando qualidade dos chunks...")

from chunking_utils import validate_chunks
from collections import defaultdict

# Validar chunks gerados
validation_result = validate_chunks(chunking_config["output_dir"])

if validation_result["valid"]:
    print("\n✅ Validação bem-sucedida")
    print(f"   📄 Arquivos de chunks: {validation_result['chunk_files_count']}")
    print(f"   🔪 Total de chunks: {validation_result['total_chunks']:,}")
    print(f"   🎯 Total de tokens: {validation_result['total_tokens']:,}")
    print(f"   📏 Média tokens/chunk: {validation_result['avg_tokens_per_chunk']:.0f}")
else:
    print(f"\n❌ Problema na validação: {validation_result.get('error', 'Erro desconhecido')}")

# Análise detalhada de qualidade
chunks_dir = Path(chunking_config["output_dir"])
chunk_files = list(chunks_dir.glob("*_chunks.json"))

quality_stats = {
    "size_distribution": defaultdict(int),
    "total_chunks": 0,
    "total_tokens": 0,
    "min_chunk_size": float('inf'),
    "max_chunk_size": 0,
    "chunks_with_overlap": 0,
    "empty_chunks": 0
}

print(f"\n📈 Análise detalhada de {len(chunk_files)} arquivos de chunks:")

for chunk_file in chunk_files:
    try:
        with open(chunk_file, 'r', encoding='utf-8') as f:
            chunk_data = json.load(f)
        
        chunks = chunk_data.get("chunks", [])
        
        for chunk in chunks:
            token_count = chunk.get("token_count", 0)
            has_overlap = chunk.get("has_overlap", False)
            text = chunk.get("text", "")
            
            quality_stats["total_chunks"] += 1
            quality_stats["total_tokens"] += token_count
            
            # Distribuição por tamanho
            size_bucket = (token_count // 100) * 100  # Buckets de 100 tokens
            quality_stats["size_distribution"][size_bucket] += 1
            
            # Min/Max
            quality_stats["min_chunk_size"] = min(quality_stats["min_chunk_size"], token_count)
            quality_stats["max_chunk_size"] = max(quality_stats["max_chunk_size"], token_count)
            
            # Overlap
            if has_overlap:
                quality_stats["chunks_with_overlap"] += 1
            
            # Chunks vazios
            if not text.strip():
                quality_stats["empty_chunks"] += 1
                
    except Exception as e:
        print(f"   ⚠️ Erro ao analisar {chunk_file.name}: {e}")

# Estatísticas de qualidade
if quality_stats["total_chunks"] > 0:
    avg_size = quality_stats["total_tokens"] / quality_stats["total_chunks"]
    overlap_percentage = (quality_stats["chunks_with_overlap"] / quality_stats["total_chunks"]) * 100
    
    print(f"\n📏 Estatísticas de tamanho:")
    print(f"   📊 Chunks totais: {quality_stats['total_chunks']:,}")
    print(f"   📐 Tamanho médio: {avg_size:.0f} tokens")
    print(f"   📉 Menor chunk: {quality_stats['min_chunk_size']} tokens")
    print(f"   📈 Maior chunk: {quality_stats['max_chunk_size']} tokens")
    
    print(f"\n🔗 Análise de overlap:")
    print(f"   Chunks com overlap: {quality_stats['chunks_with_overlap']:,} ({overlap_percentage:.1f}%)")
    
    if quality_stats["empty_chunks"] > 0:
        print(f"\n⚠️ Problemas encontrados:")
        print(f"   Chunks vazios: {quality_stats['empty_chunks']}")
    
    print(f"\n📊 Distribuição por tamanho:")
    for size_bucket in sorted(quality_stats["size_distribution"].keys()):
        count = quality_stats["size_distribution"][size_bucket]
        percentage = (count / quality_stats["total_chunks"]) * 100
        print(f"   {size_bucket}-{size_bucket+99} tokens: {count:,} chunks ({percentage:.1f}%)")

# Preparar lista de arquivos de chunks para próxima etapa
chunk_files_info = []
for result in successful_chunking:
    chunk_files_info.append({
        "source_document": result["source_document"],
        "chunks_file": result["chunks_file"],
        "chunk_count": result["chunk_count"],
        "total_tokens": result["total_tokens"]
    })

print(f"\n📋 Arquivos de chunks preparados para próxima etapa: {len(chunk_files_info)}")
print("✅ Validação concluída!")

## 💾 Salvamento dos Resultados

In [ ]:
# 💾 SALVAMENTO DOS RESULTADOS PARA PRÓXIMA ETAPA
print("💾 Salvando resultados da segmentação...")

from pipeline_utils import get_timestamp

# Preparar dados de saída para próxima etapa
stage_output = {
    "stage_info": {
        "stage_number": 4,
        "stage_name": "chunking",
        "completed_at": get_timestamp(),
        "status": "success" if len(successful_chunking) > 0 else "completed_with_warnings",
        "chunking_duration_seconds": chunking_duration
    },
    
    "chunking_config": chunking_config,
    
    "input_files": {
        "total_input_files": len(text_files),
        "available_files": len(available_text_files),
        "processed_files": len(chunking_results)
    },
    
    "chunk_files": chunk_files_info,
    
    "chunking_results": {
        "successful_documents": len(successful_chunking),
        "failed_documents": len(failed_chunking),
        "total_chunks_created": total_chunks,
        "total_tokens_processed": total_tokens
    },
    
    "quality_analysis": {
        "total_chunks": quality_stats["total_chunks"],
        "avg_tokens_per_chunk": quality_stats["total_tokens"] / quality_stats["total_chunks"] if quality_stats["total_chunks"] > 0 else 0,
        "min_chunk_size": quality_stats["min_chunk_size"] if quality_stats["min_chunk_size"] != float('inf') else 0,
        "max_chunk_size": quality_stats["max_chunk_size"],
        "chunks_with_overlap": quality_stats["chunks_with_overlap"],
        "empty_chunks": quality_stats["empty_chunks"],
        "size_distribution": dict(quality_stats["size_distribution"])
    },
    
    "statistics": {
        "documents_chunked": len(successful_chunking),
        "total_chunks": total_chunks,
        "total_tokens": total_tokens,
        "avg_chunks_per_document": total_chunks / len(successful_chunking) if successful_chunking else 0,
        "avg_tokens_per_chunk": total_tokens / total_chunks if total_chunks > 0 else 0,
        "chunking_time_seconds": chunking_duration,
        "chunk_files_generated": len(chunk_files)
    },
    
    "errors": [
        {
            "source_document": result["source_document"],
            "error": result["error"]
        }
        for result in failed_chunking
    ],
    
    "next_stage_instructions": {
        "chunks_directory": chunking_config["output_dir"],
        "chunk_files_to_embed": [info["chunks_file"] for info in chunk_files_info],
        "total_chunks_to_process": total_chunks,
        "recommended_embedding_model": "BAAI/bge-m3",
        "recommended_batch_size": 32
    }
}

# Salvar usando PipelineState
state = PipelineState()
state.save_stage_data(4, stage_output)

# Marcar etapa como concluída
state.mark_stage_completed(4)

print(f"✅ Resultados salvos: {state.metadata_dir / 'stage_04_chunking.json'}")
print(f"✅ Checkpoint criado: {state.checkpoints_dir / 'stage_04_completed.lock'}")

# Salvar relatório detalhado de chunking
chunking_report_path = Path(chunking_config["output_dir"]) / "chunking_report.json"
with open(chunking_report_path, 'w', encoding='utf-8') as f:
    json.dump({
        "timestamp": get_timestamp(),
        "config": chunking_config,
        "results": chunking_results,
        "quality_stats": quality_stats,
        "validation": validation_result
    }, f, indent=2, ensure_ascii=False, default=str)

print(f"📝 Relatório detalhado salvo: {chunking_report_path}")

# Exibir resumo final
print("\n" + "=" * 60)
print("🎉 ETAPA 4 CONCLUÍDA COM SUCESSO!")
print("=" * 60)

print(f"📊 Resumo da Segmentação:")
print(f"   📄 Documentos segmentados: {len(successful_chunking)}/{len(available_text_files)}")
print(f"   🔪 Chunks criados: {total_chunks:,}")
print(f"   🎯 Tokens processados: {total_tokens:,}")
print(f"   📏 Tamanho médio: {total_tokens/total_chunks:.0f} tokens/chunk" if total_chunks > 0 else "   📏 Tamanho médio: N/A")
print(f"   ⏱️ Tempo total: {chunking_duration:.1f} segundos")
print(f"   📍 Localização: {chunking_config['output_dir']}")

if failed_chunking:
    print(f"   ⚠️ Avisos: {len(failed_chunking)} documentos falharam")
else:
    print(f"   ✅ Status: Todos os documentos segmentados com sucesso")

print(f"\n🚀 Pronto para próxima etapa: 05_GERACAO_EMBEDDINGS.ipynb")
print(f"📋 {total_chunks:,} chunks prontos para geração de embeddings")

# Exibir progresso do pipeline
print("\n📈 Progresso do Pipeline:")
show_pipeline_progress()

---

## ✅ **Etapa 4 Concluída!**

### 🎯 **O que foi feito:**
- ✅ Texto segmentado em chunks de tamanho otimizado
- ✅ Overlap preservado para manter contexto
- ✅ Limites de frases respeitados (sem cortes abruptos)
- ✅ Qualidade validada e estatísticas coletadas
- ✅ Chunks organizados e catalogados

### 🚀 **Próxima etapa:**
**`05_GERACAO_EMBEDDINGS.ipynb`** - Geração de vetores com BAAI/bge-m3

### 📊 **Arquivos gerados:**
- `pipeline_data/chunks/` - Chunks organizados por documento
- `pipeline_data/chunks/*_chunks.json` - Chunks com metadados completos
- `pipeline_data/metadata/stage_04_chunking.json` - Estatísticas e configuração
- `pipeline_data/chunks/chunking_report.json` - Relatório detalhado de qualidade
- `pipeline_data/checkpoints/stage_04_completed.lock` - Checkpoint de conclusão

### 📋 **Dados para próxima etapa:**
- **Chunks otimizados:** Prontos para geração de embeddings
- **Metadados preservados:** Origem, posição, overlap
- **Distribuição balanceada:** Tamanhos adequados para modelos de embedding

### 🔧 **Para executar próxima etapa:**
1. Abra o notebook `05_GERACAO_EMBEDDINGS.ipynb`
2. Execute as células em sequência
3. Ou use o `00_PIPELINE_MASTER.ipynb` para execução automática

---

**🔪 Texto segmentado inteligentemente! Pronto para vetorização.**